# CodeLab 0904 - Guidance

Guidance란?

Microsoft가 올 해 5월에 발표한 새로운 language로 LLM을 제어할 수 있다. OpenAI 모델, huggingface 모델들과 높은 호환성을 보인다.

대화를 디자인하고 언어모델이 text를 처리하는 흐름을 컨트롤 할 수 있는 더 단순하고 직관적이면서도 rigour한(엄격한) 방법을 제시한다.

LangChain은 과하게 encapsulated 되어 변형의 여지를 허락하지 않지만, guidance는 덜 독단적이고 유연하다. (비슷한 유형의 모듈)

현재 Pre-release 상태로 아직 완성되지 않은 기능이 많은 듯

- 기존 프롬프트보다 더 효과적으로 언어 모델을 제어할 수 있는 언어.  
- Handlebars 템플릿 기반으로 출력 구조와 방법을 지정하고, 논리적 흐름을 제어할 수 있음. ( {{변수}}형태의 템플릿언어 )
- 프롬프트 내부에서 키/값 캐시를 사용하고, 원하는 부분만 추론하도록 하여 추론 성능 향상.  
- 단순한 출력 구조 제어를 넘어 JSON 등 유효한 구문 보장까지 가능.  
- HuggingFace 모델과 손쉬운 통합을 지원.

In [3]:
import sys
sys.path.append("/Workspace/ky/SharedTask2023/src")
from model_dict import *
from inference import *
import pandas as pd

import guidance

/opt/conda/envs/Eval4NLP23/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# model은 AutoModelForCausalLM, LlamaForCausalLM, AutoGPTQForCausalLM 등의 transformers models
# tokenizer도 마찬가지로 transfomers 라이브러리 내 tokenizer
# load_from_catalogue()는 내부 인자에서 위에서 언급한 모델과 해당 모델이 학습될 때 사용된 input_prompt, assistant_prompt등을 함께 반환
# "user_prompt": "### Instruction:", "### Human:", "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.\n\n USER: ",
# "assistant_prompt": "### Response:", "### Assistant:"
# 학습할 때 사용한 frame을 그대로 사용하는 것이 좋다고 함

model_name = 'orca_mini_v3_7b'
model_path = f'/Workspace/jh/SharedTask2023/models/{model_name}'

model, tokenizer, u_prompt, a_prompt = load_from_catalogue(model_name, model_path, "cuda:1")

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.14s/it]


In [5]:
g_model = guidance.llms.Transformers(
            model, tokenizer=tokenizer, trust_remote_code=True,
)
guidance.llms.Transformers.cache.clear()
guidance.llm = g_model

## Program 선언

기본적으로 guidance()내에 prompt가 위치하게 되고 이러한 형태를 program이라 칭함. (string 형태)



In [5]:
program = guidance('''What is {{example}}?''')

# 실행되지 않은 program은 입력한 prompt를 품고 있음
# guidance 내의 프롬프트는 일반적인 string부분과 {{내용}}로 감싸진 변수 or 명령 부분으로 구분할 수 있음
# 위의 예시에서 'What is'는 string 부분이고, {{example}}은 example이라는 이름의 변수를 받을 수 있는 placeholder
# 변수명은 지정하기 나름임
program 

Stop program What is {{example}} ?

In [120]:
# 생성한 program을 지정한 변수를 삽입하여 실행
executed_program = program(example='crimama')()

Stop program What is crimama ?

In [121]:
# program에서 사용된 변수들은 executed_program에서 반환됨
executed_program['example']

'crimama'

## 생성 및 선택

### 생성 명령어와 temperature

In [129]:
# {{gen}} command를 사용하여 LM에게 생성을 지시할 수 있음
# {{gen "변수명"}}을 사용할 시 생성된 답변은 지정한 변수명을 가진 변수에 저장됨
# 생성하고자 하는 내용을 "변수명"에 적는 것이 아님!

program = guidance('''
{{a_prompt}}
The meaning of "{{name}}" is {{gen 'meaning_of_name'}}
''')
executed_program = program(name='crimama', a_prompt=a_prompt)()

Stop program 
 ### Assistant: 
The meaning of " crimama " is not clear. It could be a misspelling or a slang term that is not well-known. However, it is possible that it could be a combination of "criminal" and "drama." In this context, "crimama" might refer to a story or situation that involves criminal activities and has dramatic elements.

In [130]:
executed_program['meaning_of_name']

'not clear. It could be a misspelling or a slang term that is not well-known. However, it is possible that it could be a combination of "criminal" and "drama." In this context, "crimama" might refer to a story or situation that involves criminal activities and has dramatic elements.'

In [131]:
# {{gen}} 내부에 stop 인자를 넣어 특정 string 출현 시 종료시킬 수 있다.
# guidance의 큰 장점 중 하나가 caching을 통해 반복적인 생성을 최소화하는 것이다.
# 동일한 프롬프트 입력 시, 기존의 답변을 참조하여 재사용할 수 있다.
# 이 때 temperature 인자의 값이 0(default)라면 더 이상 답변을 샘플링하지 않고 기존의 답변을 그대로 사용한다.
# temperature가 0인채로 복수답변을 생성할 경우 모든 값이 동일하게 출력된다.

program = guidance('''
{{a_prompt}}
The meaning of "{{name}}" is {{gen 'meaning_of_name' temperature=0}}
''')
executed_program = program(name='crimama', a_prompt=a_prompt)()


Stop program 
 ### Assistant: 
The meaning of " crimama " is not clear. It could be a misspelling or a slang term that is not well-known. However, it is possible that it could be a combination of "criminal" and "drama." In this context, "crimama" might refer to a story or situation that involves criminal activities and has dramatic elements.

In [132]:
# temperature가 0이 아니라면, LLM이 샘플링을 수행한다.
# 복수 생성의 경우 각 답변이 새로이 샘플링되어 출력된다.

program = guidance('''
{{a_prompt}}
The meaning of "{{name}}" is {{gen 'meaning_of_name' temperature=1}}
''')
executed_program = program(name='crimama', a_prompt=a_prompt)()


Stop program 
 ### Assistant: 
The meaning of " crimama " is not clear. However, it could be a misspelling or a non-literal term that means something related to crime or dangerous situations. Without more context, it's difficult to determine the exact meaning.

In [ ]:
# https://github.com/guidance-ai/guidance/blob/main/guidance/library/_gen.py#L116
# temperature는 0이 아니라면 차이가 없다.. 

# set the cache seed to 0 if temperature is 0
if temperature > 0:
    cache_seed = parser.program.cache_seed
    parser.program.cache_seed += 1
else:
    cache_seed = 0

### 종료인자

In [6]:
# guidance는 생성 종료 명령어로 stop, stop_regex 등을 지원한다.
# stop 인자 내에 단일 string 혹은 list 형태를 입력하여 해당 string 출력 시 생성을 종료할 수 있다. 
# stop_regex는 정규표현식을 지원하는 종료 명령어다.

program = guidance('''
{{a_prompt}}
The meaning of "{{name}}" is {{gen 'meaning_of_name' stop='.' temperature=0}}
''')
executed_program = program(name='crimama', a_prompt=a_prompt)()


Stop program 
 ### Assistant: 
The meaning of " crimama " is not clear

### 질의응답

In [12]:
# u_prompt, a_prompt: user prompt, assistant prompt의 줄임말로 실제 학습에 이용된 구조이다.  
# 이 구조를 inference 시 재사용하여 가장 안정적으로 원하는 답변을 생성할 수 있다.
# {{gen}} 명령어에 이전에 생성하고자 하는 text에 대한 구체적인 명령을 할 수 있음


program = guidance('''
{{u_prompt}}
Generate 5 possible answer very concisely.
Remember each answer should be no longer than 1 sentence!!!

{{a_prompt}}
Possible meaning of "{{name}}": {{gen 'meaning_of_name' temperature=0.3}}
''')
executed_program = program(name='crimama',
                           u_prompt=u_prompt,
                           a_prompt=a_prompt)()

Stop program 
 ### System: You are an AI assistant that follows instruction extremely well. Help as much as you can. 

 ### User: 
Generate 5 possible answer very concisely.
Remember each answer should be no longer than 1 sentences!!!

 ### Assistant: 
Possible meaning of " crimama ": 
1. A slang term for a fun and exciting crime-solving adventure.
2. A combination of "criminal" and "drama," representing a thrilling storyline involving crime and drama elements.
3. An abbreviation for "criminal mama," possibly referring to a powerful or influential mother figure involved in criminal activities.
4. A playful term for a crime-related event or situation.
5. A potential misspelling of "criminally" or "criminally," emphasizing the illegal or wrongdoing aspect of the subject.

### 복수 답변 생성

In [84]:
# {{gen}} 내부인자로 n을 선언하여 복수답변을 생성할 수 있음.

program = guidance('''
{{u_prompt}}
Generate possible answer very concisely in one sentence!

{{a_prompt}}
Possible meaning of "{{name}}": {{gen 'meaning_of_name' temperature=1 n=3}}
''')
executed_program = program(name='crimama',
                           u_prompt=u_prompt,
                           a_prompt=a_prompt)()

Stop program 
 ### System: You are an AI assistant that follows instruction extremely well. Help as much as you can. 

 ### User: 
Generate possible answer very concisely in one sentence!

 ### Assistant: 
Possible meaning of " crimama ": <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_bd7a08309c744be19c7bfa60a746ce6a(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("bd7a08309c744be19c7bfa60a746ce6a_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("bd7a08309c744be19c7bfa60a746ce6a_" + (i+1)); if (!next_el) { next_el = document.getElementById("bd7a08309c744be19c7bfa60a746ce6a_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 3) + 1) + "/" + 3;}cycle_bd7a08309c744be19c7bfa60a746ce6a(this);'>1/3 An expression possibly used to indicate that something is extremely fun, thrilling, or exciting. An expression used to represent someone who is skilled in a particular field or occupation. It is likely a combination of "criminal" and "mama," possibly suggesting the impact or involvement of a mother in a criminal context.

In [85]:
# 복수 생성한 답변은 리스트 형태로 저장됨
executed_program['meaning_of_name']

['An expression possibly used to indicate that something is extremely fun, thrilling, or exciting.',
 'An expression used to represent someone who is skilled in a particular field or occupation.',
 'It is likely a combination of "criminal" and "mama," possibly suggesting the impact or involvement of a mother in a criminal context.']

In [86]:
# you can flush a cache by calling the clear method
# (this returns the number of items that were cleared)
guidance.llm.cache.clear()

# you can also disable caching by passing caching=False to the LLM constructor
# guidance.llm = guidance.llms.OpenAI("text-davinci-003", caching=False)

### 언어모델의 답변을 정규표현식으로 제약걸기

In [11]:
# program 내부 인자로 프롬프트 전체를 받을 수 있음 -> 프롬프트 관리에 용이

# {{gen}} 내 인자로 pattern 선언하여 생성을 주어진 정규표현식 내로 한정할 수 있음


prompt_example = """
{{u_prompt}}
Is the word '{{name}}' good for nickname?
Score your answer from 1 to 5.
Score of 5 means it's perfect for nickname.
Score of 1 means it's terrible for nickname.

{{a_prompt}}
Select: {{gen "score" pattern='[1-5]'}}
"""

program = guidance("{{input_prompt}}")

executed_program = program(input_prompt=prompt_example,
                           u_prompt=u_prompt,
                           a_prompt=a_prompt,
                           name='tootouch',)()

Stop program 
 ### System: You are an AI assistant that follows instruction extremely well. Help as much as you can. 

 ### User: 
Is the word ' tootouch ' good for nickname?
Score your answer from 1 to 5.
Score of 5 means it's perfect for nickname.
Score of 1 means it's terrible for nickname.

 ### Assistant: 
Select: 2

### 선택하기

In [12]:
# {{select}} 내 인자로 options를 선언하여 선택 명령에 대한 옵션을 선언, 리스트의 형태로 선택가능한 옵션을 제공할 수 있다.
# {{select}} command를 사용하여 LLM이 주어진 옵션 중에 하나를 선택하게 할 수 있음


select_options = ['Yes', 'Maybe', 'No']

prompt_example = """
{{u_prompt}}
Is the word '{{name}}' good for nickname?
Select among "Yes", "No", or "Maybe".

{{a_prompt}}
Select: {{select "selection" logprobs='logprobs' options=select_options}}
"""


program = guidance("{{input_prompt}}")

executed_program = program(input_prompt=prompt_example,
                           u_prompt=u_prompt,
                           a_prompt=a_prompt,
                           name='tootouch',
                           select_options=select_options)()

Stop program 
 ### System: You are an AI assistant that follows instruction extremely well. Help as much as you can. 

 ### User: 
Is the word ' tootouch ' good for nickname?
Select among "Yes", "No", or "Maybe".

 ### Assistant: 
Select: Maybe

In [13]:
# 선택 명령에 대한 confidence를 log probability로 리턴함
executed_program['logprobs']

{'Yes': -1.9694147109985352,
 'Maybe': -0.7194147706031799,
 'No': -0.9850397706031799}

In [20]:
# 언어모델의 답변을 그에 해당하는 수치(Yes:5, Maybe:3, No:1)와 확률과 곱하여 사용할 수도 있다. 추후 설명 예정
np.sum(np.exp(list(executed_program['logprobs'].values()))*np.array([5,3,1]))

2.532228491860184

In [128]:
guidance.llm.cache.clear()
torch.cuda.empty_cache()

### 단일 프롬프트 내에서 질문과 생성 반복하기

In [22]:
# guidance의 큰 장점 중 하나로, 단일한 프롬프트 내에 복수의 질의응답을 수행할 수 있고,
# caching으로 인해 모든 과정이 서로 영향을 끼친다.


select_options = ['Yes', 'Maybe', 'No']

prompt_example = """
{{u_prompt}}
Answer a few possible meanings of given word.
Given word: {{name}}
{{a_prompt}}
Answer:{{gen "answer" temperature=1}}


Is the word '{{name}}' good for nickname?
And based on your previous answer, Select your answer among "Yes", "No", or "Maybe".
{{a_prompt}}
Select: {{select "selection" logprobs='logprobs' options=select_options}}
"""


program = guidance("{{input_prompt}}")

executed_program = program(input_prompt=prompt_example,
                           u_prompt=u_prompt,
                           a_prompt=a_prompt,
                           name='tootouch',
                           select_options=select_options)()

Stop program 
 ### System: You are an AI assistant that follows instruction extremely well. Help as much as you can. 

 ### User: 
Answer a few possible meanings of given word.
Given word: tootouch 
 ### Assistant: 
Answer: tootouch can have several meanings depending on the context, but some possible meanings include:

1. In music, "tootouch" could refer to a percussive action or a technique used to create a specific sound on a musical instrument, like "tooting" a horn or "touching" a drum.

2. In cooking, "tootouch" might be a slang term for a small portion or serving size of a food item, similar to "alittle" or "a pinch."

3. In technology, "tootouch" could be short for "too touch screen" or "too touch friendly," referring to an electronic device, like a smartphone, that is easy to interact with using just a touch.

4. In nature, "tootouch" might be a colloquial expression for something that is lightly brushing against or gently touching another object or being.

5. In sports, "tootouch" could be used to describe a minor collision or light contact between players during a game. 


Is the word ' tootouch ' good for nickname?
And based on your previous answer, Select your answer among "Yes", "No", or "Maybe".
 ### Assistant: 
Select: Maybe

In [26]:
# 스스로 생성한 답변이 이후 답변에 영향을 준다.
print(executed_program['logprobs'])
print(np.sum(np.exp(list(executed_program['logprobs'].values()))*np.array([5,3,1])))

{'Yes': -1.4747332334518433, 'Maybe': -0.30285826325416565, 'No': -3.427858352661133}
3.39276657626042


In [25]:
guidance.llm.cache.clear()
torch.cuda.empty_cache()

In [27]:
# 생성한 답변을 동일 프롬프트 내에서 명시적으로 호출할 수 있다.

select_options = ['Yes', 'Maybe', 'No']

prompt_example = """
{{u_prompt}}
Answer some possible creative meanings of given word.
Given word: {{name}}
{{a_prompt}}
Answer:{{gen "answer" temperature=1}}


Is the word '{{name}}' good for nickname?
Based on your previous answer "{{answer}}", 
Select your answer among "Yes", "No", or "Maybe".
{{a_prompt}}
Select: {{select "selection" logprobs='logprobs' options=select_options}}
"""


program = guidance("{{input_prompt}}")

executed_program = program(input_prompt=prompt_example,
                           u_prompt=u_prompt,
                           a_prompt=a_prompt,
                           name='crimama',
                           select_options=select_options)()

Stop program 
 ### System: You are an AI assistant that follows instruction extremely well. Help as much as you can. 

 ### User: 
Answer some possible creative meanings of given word.
Given word: crimama 
 ### Assistant: 
Answer: Crimama could potentially be a combination of words that represents a thrilling or adventurous activity or a location. Below are some possible interpretations:

1. Crimama: City of Criminals - A city known for its high crime rate or a location that attracts people who are interested in exploring criminal elements.
2. Crimama: Climbing Rainforest Adventure Madness - An activity or location that involves a challenging and exciting climb through a rainforest, possibly with a combination of elements such as trekking, rock climbing, and exploration.
3. Crimama: Crime and Martial Arts - A term that embodies both the world of crime and martial arts and their interconnectedness, possibly highlighting a place or event where these two concepts converge or a genre of fiction that combines crime and martial arts elements. 


Is the word ' crimama ' good for nickname?
Based on your previous answer " Crimama could potentially be a combination of words that represents a thrilling or adventurous activity or a location. Below are some possible interpretations:

1. Crimama: City of Criminals - A city known for its high crime rate or a location that attracts people who are interested in exploring criminal elements.
2. Crimama: Climbing Rainforest Adventure Madness - An activity or location that involves a challenging and exciting climb through a rainforest, possibly with a combination of elements such as trekking, rock climbing, and exploration.
3. Crimama: Crime and Martial Arts - A term that embodies both the world of crime and martial arts and their interconnectedness, possibly highlighting a place or event where these two concepts converge or a genre of fiction that combines crime and martial arts elements. ", 
Select your answer among "Yes", "No", or "Maybe".
 ### Assistant: 
Select: Yes

# Eval4NLP

특정한 document에 대해서, 서로 다른 크기와 구조를 가진 언어모델이 생성한 summary들을 공개 (데이터셋 종류 3가지: train, dev, test)  
모든 summary들을 실제 사람이 여러 가지 척도에 의하여 생성한 ground truth score가 존재함 (1-5) (train 데이터셋에 대하여 gt_score 공개)

workshop의 최종 목표는 언어모델을 freeze한 상태로 "오직 프롬프트"만 이용하여 human evaluation과 유사한 점수를 생성케 하는 것  
Objective: the correlation(kendall) between human evaluation and prompted score.

train 데이터셋은 자유롭게 이용  
dev 데이터셋은 leaderboard에 순위를 매기는데 사용(최종 등수 아님)  
test 데이터셋이 최종적인 순위를 가리는데 사용된다.

## 프롬프팅 전략

### 1. LM가 보유한 추론 능력을 활용하기 위해, 가장 먼저 LM이 스스로 주어진 원본 문장을 요약하게 한다.</br>-> 리스트 형태로 반환 (aspects)

In [6]:
summ_path = '/Workspace/jh/SharedTask2023/data/summarization/train_summarization.tsv'
summ_df = pd.read_csv(summ_path, delimiter="\t", quoting=csv.QUOTE_NONE)
SRC = summ_df['SRC']
HYP = summ_df['HYP']

In [12]:
idx = 4

In [13]:
# Source document #4
print(SRC[idx])

The man who allegedly killed his neighbour was believed to be suffering from dementia and alcohol abuse at the time of the horrendous murder in 2012, a jury has heard. Luigi Costa, 71, is accused of stomping on his elderly neighbour Terrence Freebody's head, cutting his throat and stabbing him multiple times in the dining room of his home on Mugga Way in Red Hill, Canberra on July 2012. Forensic psychiatrist Professor Paul Mullen examined Costa after the attack and believes there was evidence of the accused's state of mind declining in the lead-up to the incident and also during the event, The ABC reported. Luigi Costa, 71, is accused of killing his elderly neighbour Terrence Freebody in the dining room of his home on Mugga Way, Red Hill, Canberra in July 2012 Professor Mullin told the court that the triple 0 calls which Costa made 'almost certainly' during the incident proved that his mood fluctuated from 'moment to moment'. He also said the trail of faeces from the crime scene  to th

The man who allegedly killed his neighbour was believed to be suffering from dementia and alcohol abuse at the time of the horrendous murder in 2012, a jury has heard. Luigi Costa, 71, is accused of stomping on his elderly neighbour Terrence Freebody's head, cutting his throat and stabbing him multiple times in the dining room of his home on Mugga Way in Red Hill, Canberra on July 2012. Forensic psychiatrist Professor Paul Mullen examined Costa after the attack and believes there was evidence of the accused's state of mind declining in the lead-up to the incident and also during the event, The ABC reported. Luigi Costa, 71, is accused of killing his elderly neighbour Terrence Freebody in the dining room of his home on Mugga Way, Red Hill, Canberra in July 2012 Professor Mullin told the court that the triple 0 calls which Costa made 'almost certainly' during the incident proved that his mood fluctuated from 'moment to moment'. He also said the trail of faeces from the crime scene  to the toilet were police found Costa suggested his confused mental state. 'I believe that at the time of the killing Mr Costa's brain was compromised by dementia and when you add to this alcohol that led to a confused state,' he said. 'I do not think you would understand what you were doing in that state or understand the dreadful long-term consequences.' Prosecutor Shane Drumgold suggested that Costa feigned his failing memory while being interviewed by Professor Mullin - who denied these claims. 'My view is that it was more likely than possible that Mr Costa was suffering a demented illness in the weeks, months and possibly years in the lead up to this event,' he said. Costa's state of mind is critical to case, as The jury can return a not guilty verdict  if Costa's unstable mindset at time of the attack can be proven. Sorry we are not currently accepting comments on this article.


대략적인 내용:  

평소에 치매와 알콜중독을 앓고 있던 71세의 Luigi Costa가 이웃주민을 살해했다.  

정신과의사가 검사했을 때, 사건 당시 피고인의 정신 활동이 저하되어 있었다고 한다.  

그리고 그가 법정에서 증언하길 사건 직후 000 call(응급전화)를 한 걸 보니 그의 정신상태가 오락가락했을 것이라 주장했다.

Costa의 범행 당시의 정신 상태가 사건에서 굉장히 중요하게 작용할 것이고, 의사의 말이 맞다고 증명되면 무죄가 될 수 도 있다.  

In [14]:
# LM-made summary #4
print(HYP[idx])

the man who allegedly killed his neighbour was believed to be suffering from dementia and alcohol abuse at the time of the horrendous murder in 2012 , a jury has heard . luigi costa , 71 , is accused of stomping on his elderly neighbour terrence freebody 's head , cutting his throat and stabbing him multiple times in the dining room of his home on mugga way in red hill , canberra on july 2012 . forensic psychiatrist professor paul mullen examined costa after the attack and believes there was evidence of the accused 's state of mind declining in the lead-up to the incident and also during the event , the abc reported .


the man who allegedly killed his neighbour was believed to be suffering from dementia and alcohol abuse at the time of the horrendous murder in 2012 , a jury has heard . luigi costa , 71 , is accused of stomping on his elderly neighbour terrence freebody 's head , cutting his throat and stabbing him multiple times in the dining room of his home on mugga way in red hill , canberra on july 2012 . forensic psychiatrist professor paul mullen examined costa after the attack and believes there was evidence of the accused 's state of mind declining in the lead-up to the incident and also during the event , the abc reported .

In [15]:
program = guidance('''
    {{input_prompt}}
    ''')

In [16]:
# 주어진 text에 대하여 짧은 요약문들을 생성하라
# 답변을 list형태로 주어야 하고, 각 문장들을 구분되어야 한다.
# 누가 무엇을 했는지에 집중하고, 무엇이 정말 중요하고 무엇이 중복되고 불필요한지 비판적으로 생각하라.

summarization_prompt = '''
{{prompt_placeholder}}

Generate "short" summarizations of given source text that should be considered when summarizing a document.
You need to give answer with the form of LIST. Every lines of sentences must be seperated with '\\n'!
You should focus on "who" did " "what".
You need to think very pessimistically whether it is really important or not and Delete something redundant or unnecessary.

Source text: {{gt}}

{{response_placeholder}}

Short summarizations: {{gen 'summarization' temperature=1}}
'''

In [17]:
executed_program = program(
        gt=SRC[idx],
        hyp=HYP[idx],
        input_prompt=summarization_prompt,
        prompt_placeholder=u_prompt,
        response_placeholder=a_prompt)()

Stop program 
 
 ### System: You are an AI assistant that follows instruction extremely well. Help as much as you can. 

 ### User: 

Generate "short" summarizations of given source text that should be considered when summarizing a document.
You need to give answer with the form of LIST. Every lines of sentences must be seperated with '\n'!
You should focus on "who" did " "what".
You need to think very pessimistically whether it is really important or not and Delete something redundant or unnecessary.

Source text: The man who allegedly killed his neighbour was believed to be suffering from dementia and alcohol abuse at the time of the horrendous murder in 2012, a jury has heard. Luigi Costa, 71, is accused of stomping on his elderly neighbour Terrence Freebody's head, cutting his throat and stabbing him multiple times in the dining room of his home on Mugga Way in Red Hill, Canberra on July 2012. Forensic psychiatrist Professor Paul Mullen examined Costa after the attack and believes there was evidence of the accused's state of mind declining in the lead-up to the incident and also during the event, The ABC reported. Luigi Costa, 71, is accused of killing his elderly neighbour Terrence Freebody in the dining room of his home on Mugga Way, Red Hill, Canberra in July 2012 Professor Mullin told the court that the triple 0 calls which Costa made 'almost certainly' during the incident proved that his mood fluctuated from 'moment to moment'. He also said the trail of faeces from the crime scene to the toilet were police found Costa suggested his confused mental state. 'I believe that at the time of the killing Mr Costa's brain was compromised by dementia and when you add to this alcohol that led to a confused state,' he said. 'I do not think you would understand what you were doing in that state or understand the dreadful long-term consequences.' Prosecutor Shane Drumgold suggested that Costa feigned his failing memory while being interviewed by Professor Mullin - who denied these claims. 'My view is that it was more likely than possible that Mr Costa was suffering a demented illness in the weeks, months and possibly years in the lead up to this event,' he said. Costa's state of mind is critical to case, as The jury can return a not guilty verdict if Costa's unstable mindset at time of the attack can be proven. Sorry we are not currently accepting comments on this article. 

 ### Assistant: 

Short summarizations: 

1. Alleged murderer Luigi Costa had dementia and alcohol abuse issues.
2. Costa accused of horrific 2012 murder of elderly neighbour Terrence Freebody.
3. Costa's state of mind is critical to trial, which could potentially result in not guilty verdict if unstable mindset confirmed.

### 2. 반복문으로 생성한 aspect를 한 개씩 입력하여, 더욱 간결한 버전을 생성한다.(너무 detail한 정보를 삭제하기) 그리고 원본 aspect와 간결한 버전의 aspect를 최종적으로 사용하여 LM-made summary 중에 해당 내용이 속해있는지를 5점 만점으로 평가하게 한다.


In [94]:
aspects = ['1. Alleged murderer Luigi Costa had dementia and alcohol abuse issues.',
           '2. Costa accused of horrific 2012 murder of elderly neighbour Terrence Freebody.',
           "3. Costa's state of mind is critical to trial, which could potentially result in not guilty verdict if unstable mindset confirmed."]

In [37]:
program = guidance('''
    {{input_prompt}}
    ''')

아래에서 언어모델은 명령에 의해 답변을 생성하게 되고, 주어진 aspect, 생성된 consice aspect 등을 고려하여 최종 점수를 부여하게 된다.  
이러한 흐름 속에 guidance의 장점이 드러난다.  
언어모델이 생성한 답변을 다시 고려하여 최종 답변을 생성할 때, 이 모두를 한 호흡에 이루어지게 하고 파라미터 관리를 용이하게 해준다.

In [93]:
# 

score_prompt = '''
{{prompt_placeholder}}
You will be given a summary and a aspect.
Read the summary and the aspect carefully.

Summary: {{summary}}
Aspect: {{aspect}}

{{response_placeholder}}

Generate a more concise version of Source text as breif as possible in one sentence.
You should focus on "who" did " "what" and delete unnecessary words.
Delete something that too detailed or redundant.
Refine the language used in the sentence with more general words.
Concise version must not be longer than original text.


Concise version of Aspect: {{gen 'explain' temperature=1}}


Do you think the summary has any similar context of "{{explain}},{{aspect}}", even roughly?
Generate your Answer carefully.
Answer: {{gen 'answer' temperature=1}}


Based on your answer "{{answer}}", score the summary from 1 to 5. 
If the summary doesn't have similar meaning, score 1.
If the summary has similar meaning, score 5.

Score: {{gen 'score' pattern='[1-5]' temperature=0}}
'''

In [96]:
# 각 aspect를 독립적으로 scoring하기 위해, 한 사이클에 한 가지 aspect에 대해 점수를 매긴다.

results = {'summary':[],
           'aspect':[],
           'explain':[],
           'answer':[],
           'score':[]}

for aspect in aspects:
    guidance.llm.cache.clear()
    torch.cuda.empty_cache()

    executed_program = program(
            summary=HYP[idx],
            aspect=aspect,
            input_prompt=score_prompt,
            prompt_placeholder=u_prompt,
            response_placeholder=a_prompt,
            silent=False)()

    var = executed_program.variables()
    results['summary'].append(var['summary'])
    results['aspect'].append(var['aspect'])
    results['explain'].append(var['explain'])
    results['answer'].append(var['answer'])
    results['score'].append(int(var['score']))

Stop program 
 
 ### System: You are an AI assistant that follows instruction extremely well. Help as much as you can. 

 ### User: 
You will be given a summary and a aspect.
Read the summary and the aspect carefully.

Summary: the man who allegedly killed his neighbour was believed to be suffering from dementia and alcohol abuse at the time of the horrendous murder in 2012 , a jury has heard . luigi costa , 71 , is accused of stomping on his elderly neighbour terrence freebody 's head , cutting his throat and stabbing him multiple times in the dining room of his home on mugga way in red hill , canberra on july 2012 . forensic psychiatrist professor paul mullen examined costa after the attack and believes there was evidence of the accused 's state of mind declining in the lead-up to the incident and also during the event , the abc reported . 
Aspect: 3. Costa's state of mind is critical to trial, which could potentially result in not guilty verdict if unstable mindset confirmed. 

 ### Assistant: 

Generate a more concise version of Source text as breif as possible in one sentence.
You should focus on "who" did " "what" and delete unnecessary words.
Delete something that too detailed or redundant.
Refine the language used in the sentence with more general words.
Concise version must not be longer than original text.


Concise version of Aspect: The court case focuses on evaluating Luigi Costa's mental state, which could lead to an acquittal if his mental instability is proven. 


Do you think the summary has any similar context of " The court case focuses on evaluating Luigi Costa's mental state, which could lead to an acquittal if his mental instability is proven. , 3. Costa's state of mind is critical to trial, which could potentially result in not guilty verdict if unstable mindset confirmed. ", even roughly?
Generate your Answer carefully.
Answer: Yes, the summary has a similar context to the provided answer. 


Based on your answer " Yes, the summary has a similar context to the provided answer. ", score the summary from 1 to 5. 
If the summary doesn't have similar meaning, score 1.
If the summary has similar meaning, score 5.

Score: 4

In [97]:
results

{'summary': ["the man who allegedly killed his neighbour was believed to be suffering from dementia and alcohol abuse at the time of the horrendous murder in 2012 , a jury has heard . luigi costa , 71 , is accused of stomping on his elderly neighbour terrence freebody 's head , cutting his throat and stabbing him multiple times in the dining room of his home on mugga way in red hill , canberra on july 2012 . forensic psychiatrist professor paul mullen examined costa after the attack and believes there was evidence of the accused 's state of mind declining in the lead-up to the incident and also during the event , the abc reported .",
  "the man who allegedly killed his neighbour was believed to be suffering from dementia and alcohol abuse at the time of the horrendous murder in 2012 , a jury has heard . luigi costa , 71 , is accused of stomping on his elderly neighbour terrence freebody 's head , cutting his throat and stabbing him multiple times in the dining room of his home on mugga

*원본 aspect보다 더 간결한 버전을 생성하라고 했는데 이해를 잘 하지 못한다.

In [101]:
# score의 평균
np.mean(results['score'])

4.666666666666667

In [100]:
# gt_score
summ_df['Score'][idx]

4.833333333325

# Conclusion

## 실험결과

baseline: 한 프롬프트에서 원본 문서와 given summary가 주어지고 곧바로 5점 만점으로 평가하게 하는 방식  
correlation: 0.2

고안한 방식  
correlation: 0.06

## 결과분석

human evaluation과의 correlation이 낮게 나온 이유
1. 사람이 요약의 퀄리티를 평가할 때, 디테일한 요소에 집중하는 정도가 사람마다 다르다.  
ex) luigi costa의 또 다른 요약문에서는 치매, 알콜 중독에 대한 언급이 없고 단순히 살인사건만 다뤄도 4.5점을 부여했다.

2. 언어모델에게 자율적으로 요약을 시키면 그 디테일한 정도의 편차가 매우 크다.  
ex) 위의 예시에서 언어모델은 스스로 3줄의 요약을 만들었지만, 다른 문서의 경우 10줄이 넘는 요약을 만들기도 한다.

3. 스스로 만든 요약을 더 간결하게 변환하여 바꾸고 평가에 사용하여 더 general한 차원의 평가를 만들고 싶었지만, 불러온 언어모델마다 명령 이해의 정도가 달라 원하는대로 되지 않았다.

어려웠던 점  
1. 동일한 의미를 주고 싶어도 쓰는 단어와 어순 등에 따라서 성능의 fluctuation이 너무 컸다. 
2. guidance 모듈의 caching을 완전히 이해하기 어렵다.  
*Eval4NLP workshop에서도 guidance를 사용하길 권장하지만 cache로 인한 memory leak 문제에 대해 아래와 같은 해결방법을 제안한다.  

In [102]:
BPG = None

.
.
.

for s, h in tqdm(data[["SRC","HYP"]].values.tolist(), desc=args.task + " progress: "):
    # Ugly fix for memory leak; perhaps with the guidance module
    if BPG:
        del BPG
    BPG = DirectAssessment(model=model, tokenizer=tokenizer, prompt=prompt)

SyntaxError: invalid syntax (902122092.py, line 3)

향후 방향성  
1. 프롬프트 자동 생성 논문들에서 제시한 방법을 사용해 원하는 기능을 가장 잘 수행할 수 있는 프롬프트를 스스로 만들게 할 것
2. 중요한 내용을 모두 포함하면서도 최대한 간결한 요약을 생성하도록 유도하여 실제 사람이 평가하는 방식을 모사하게 할 것
3. 수행한 다른 실험에선 단순히 어떤 source text에 대해 5점짜리 요약, 3점짜리 요약, 1점짜리 요약의 예시를 one-shot으로 제시한 뒤, 그와 다른 source text, 요약을 평가하라 명령했을 때, baseline보다 훨씬 높은 correlation을 달성했다. (13B 모델 기준 baseline: 0.3, 실험결과 0.408)  
이를 통해 지시사항과 그에 해당하는 예시를 들어주는 것이 명령 이해도가 높을 것이라 생각이 들었고 이후 모든 지시사항에 예시를 들어보고자 함